Übung: Volume und mehrere Container in Pod
------------------------------------------------------------------

Diese Übung Demonstriert wie sich zwei Container innerhalb eines Pods das Verzeichnis `/usr/local/apache2/htdocs` teilen.

Der Container `apache` beinhaltet den Web Server und der Container `file-puller` schreibt alle 30 Sekunden die Datei `index.html` in das Verzeichnis `/usr/local/apache2/htdocs`.

Aus Einfachheitsgründen verwenden wir `emptyDir` als Volume.

**Erläuterungen `emptyDir`**

Das emptyDir-Volume wird angelegt, wenn ein Pod einem Node zugewiesen wird.
Alle Container in dem Pod auf diesem (Worker-)Node können dieses emptyDir (einfach
ein leeres Verzeichnis) lesen und schreiben.

Der Pfad, mit dem das emptyDir innerhalb eines Containers eingehängt wird, kann
sich innerhalb der Container des Pods unterscheiden.

Sobald ein Pod von einem Node gelöscht wird, wird auch der Inhalt des emptyDir
komplett und unwiederbringlich gelöscht. Selbst wenn der gleiche Pod auf dem gleichen
Worker Node neu erstellt wird, kann er nicht mehr auf das Volume seines Vorgängers
zugreifen. Dies bezieht sich nicht auf den Crash eines Containers des Pods.

Typische Anwendungsfälle für emptyDir-Volumes könnten sein:
* Laufzeitdaten einer Applikation (Caches), die bei der Neuerstellung des Pods ohnehin
neu erzeugt werden müssten
* Übergabe von (Laufzeit-)Konfigurationsdaten an alle Container innerhalb des
Pods

Zuerst schauen wir den Inhalt der YAML Datei an und starten dann die Ressourcen

In [ ]:
! cat 09-5-Volume/web.yaml

In [ ]:
! kubectl apply -f 09-5-Volume/web.yaml

Um das Ergebnis anschauen, holen wir uns die IP-Adresse des Loadbalancer und des Services

In [ ]:
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service web -o=jsonpath='{ .spec.ports[0].nodePort }')/"

Und geben die Erzeugten Ressourcen aus:

In [ ]:
! kubectl get pods,services
! echo "\n\nContainer"
! kubectl get pods -o=jsonpath='{range .items[*]}{"\n"}{.metadata.name}{":\t"}{range .spec.containers[*]}{.image}{", "}{end}{end}' |\
sort

Aufräumen

In [ ]:
! kubectl delete -f 09-5-Volume/web.yaml